## Imports

In [1]:
using CSV
using DataFrames
using MLJ

### import base

In [ ]:
# import base
using Pkg
Pkg.add("MLJDecisionTreeInterface")

In [11]:
all_models = models("Regression")
meta = all_models[3]
targetscitype = meta.target_scitype

AbstractVector{Continuous} (alias for AbstractArray{Continuous, 1})

In [15]:
filter_julia_classifiers(meta) =
    AbstractVector{Continuous} <: meta.target_scitype &&
    meta.is_pure_julia


models(filter_julia_classifiers)

23-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = ConstantRegressor, package_name = MLJModels, ... )
 (name = DecisionTreeRegressor, package_name = BetaML, ... )
 (name = DecisionTreeRegressor, package_name = DecisionTree, ... )
 (name = DeterministicConstantRegressor, package_name = MLJModels, ... )
 (name = ElasticNetRegressor, package_name = MLJLinearModels, ... )
 (name = EvoTreeGaussian, package_name = EvoTrees, ... )
 (name = EvoTreeRegressor, package_name = EvoTrees, ... )


In [2]:
iris = load_iris()
selectrows(iris, 1:3) |> pretty

┌──────────────┬─────────────┬──────────────┬─────────────┬──────────────────────────────────┐
│ sepal_length │ sepal_width │ petal_length │ petal_width │ target                           │
│ Float64      │ Float64     │ Float64      │ Float64     │ CategoricalValue{String, UInt32} │
│ Continuous   │ Continuous  │ Continuous   │ Continuous  │ Multiclass{3}                    │
├──────────────┼─────────────┼──────────────┼─────────────┼──────────────────────────────────┤
│ 5.1          │ 3.5         │ 1.4          │ 0.2         │ setosa                           │
│ 4.9          │ 3.0         │ 1.4          │ 0.2         │ setosa                           │
│ 4.7          │ 3.2         │ 1.3          │ 0.2         │ setosa                           │
└──────────────┴─────────────┴──────────────┴─────────────┴──────────────────────────────────┘


In [4]:
schema(iris)

┌──────────────┬───────────────┬──────────────────────────────────┐
│ names        │ scitypes      │ types                            │
├──────────────┼───────────────┼──────────────────────────────────┤
│ sepal_length │ Continuous    │ Float64                          │
│ sepal_width  │ Continuous    │ Float64                          │
│ petal_length │ Continuous    │ Float64                          │
│ petal_width  │ Continuous    │ Float64                          │
│ target       │ Multiclass{3} │ CategoricalValue{String, UInt32} │
└──────────────┴───────────────┴──────────────────────────────────┘


## Dataframe

In [4]:
import DataFrames
iris = DataFrames.DataFrame(iris)

,sepal_length,sepal_width,petal_length,petal_width,target
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [13]:
iris[!, Not(:target)]

,sepal_length,sepal_width,petal_length,petal_width
,Float64,Float64,Float64,Float64
1,5.1,3.5,1.4,0.2
2,4.9,3.0,1.4,0.2
3,4.7,3.2,1.3,0.2
4,4.6,3.1,1.5,0.2
5,5.0,3.6,1.4,0.2
6,5.4,3.9,1.7,0.4
7,4.6,3.4,1.4,0.3
8,5.0,3.4,1.5,0.2
9,4.4,2.9,1.4,0.2


In [14]:
y, X = unpack(iris, ==(:target); rng=123);

In [19]:
iris[:, :target]

150-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

## Model selection

In [24]:
Tree = @load DecisionTreeClassifier pkg=DecisionTree
tree = Tree()

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/hilsts/.julia/packages/MLJModels/kwZnx/src/loading.jl:168


DecisionTreeClassifier(
    max_depth = -1,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = 0,
    post_prune = false,
    merge_purity_threshold = 1.0,
    display_depth = 5,
    rng = Random._GLOBAL_RNG())

In [25]:
evaluate(tree, X, y,
                resampling=CV(shuffle=true),
                        measures=[log_loss, accuracy],
                        verbosity=0)

PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌────────────────────────────┬─────────────┬──────────────┬─────────────────────
│ measure                    │ measurement │ operation    │ per_fold           ⋯
├────────────────────────────┼─────────────┼──────────────┼─────────────────────
│ LogLoss(tol = 2.22045e-16) │ 2.4         │ predict      │ [2.88, 2.22e-16, 4 ⋯
│ Accuracy()                 │ 0.933       │ predict_mode │ [0.92, 1.0, 0.88,  ⋯
└────────────────────────────┴─────────────┴──────────────┴─────────────────────
                                                                1 column omitted


In [26]:
mach = machine(tree, X, y)

Machine{DecisionTreeClassifier,…} trained 0 times; caches data
  model: MLJDecisionTreeInterface.DecisionTreeClassifier
  args: 
    1:	Source @498 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @552 ⏎ `AbstractVector{Multiclass{3}}`
